In [ ]:
import json
from web3 import Web3
import web3
import csv
import array
import time
import asyncio

# contract address after deploy
myContractAddress = "0x3174c5084c196f961e310C2b7Bb8396EfEF80Da5"; 

# get abi from Data.json, get data from soini_filtered_last.csv, collect url that makes error in errorSnippet.txt
with open("./build/contracts/Data.json") as f, open('./Dataset/soini_filtered_last.csv', 'r', encoding="utf-8", errors='ignore') as csv_file, open('errorSnippet.txt', 'a') as text_file :
    info_json = json.load(f)
    abi = info_json["abi"]
    
    csv_reader = csv.reader(csv_file)
    # start at 0
    count = 0
    start = count;
    skip = 0;
    url = None
    date = None
    language = None
    projectName = None
    companyName = None
    projectPath = None
    projectAuthor = None
    transact = None
    
    # set up web3 connection with Ganache
    w3 = Web3(Web3.WebsocketProvider("ws://127.0.0.1:8545", websocket_kwargs={'timeout': 10000}))
    # read smart contract
    myContract = w3.eth.contract(address=myContractAddress, abi=abi)
    
    # set up default account to be sender
    w3.eth.defaultAccount = w3.eth.accounts[0]
    
    for row in csv_reader:
        if(count > 11465):
            break
        if(skip < start):
            skip += 1
            continue
        url = row[0]
        if row[2]:
            date = row[1]
            language = row[2]
            projectName = row[3]
            companyName = row[4]
            projectPath = row[5]
            projectAuthor = row[6]
            
            # call function to store data on blockchain                    
            try:
                transact = myContract.functions.createNewCodeSnippet(url, date, language, projectName, companyName, projectPath,projectAuthor).transact()                
                # waiting for transaction to be mined
                w3.eth.waitForTransactionReceipt(transact)
            except asyncio.TimeoutError:
                # when timeout error occured, url will be written in textfile
                text_file.write(url + " --- " + date + "\n")
                print('timeout!')                        

            # txn_receipt = w3.eth.getTransactionReceipt(transact)
            
            # collect transaction number in array
            transactionNumber = []
            transactionNumber.append(w3.toHex(transact))
#             print(transactionNumber)
#             print(txn_receipt['transactionHash'])           
        print(count)
        count += 1;
         




In [ ]:
# get code snippet's datail from url by index
# first parameter - url
# second parameter - index (start at 0)
# myContract.functions.getDetailCodeSnippet("http://stackoverflow.com/a/10001884/1136593",0).call()